In [4]:
import pandas as pd
import plotly.express as px
import ast

# Read the csv file
data = pd.read_csv('tournament_tests.csv')
data.drop(['Champion_last'], axis=1, inplace=True)

# Unique values of Config_player_best and Config_player_test
unique_values = ["{'diff': 0.875, 'edge': 0.5, 'center': 2.0, 'cohesion_diff': 0.5, 'threat_diff': 0.125, 'density_diff': 1.625}",
"{'diff': 1.125, 'edge': 0.125, 'center': 1.125, 'cohesion_diff': 0.25, 'threat_diff': 0.05, 'density_diff': 0.125}",
"{'diff': 1.125, 'edge': 1.375, 'center': 1.75, 'cohesion_diff': 0.05, 'threat_diff': 0.25, 'density_diff': 1.25}",
"{'diff': 1.25, 'edge': 0.375, 'center': 1.25, 'cohesion_diff': 0.25, 'threat_diff': 1.0, 'density_diff': 0.05}",
"{'diff': 1.25, 'edge': 0.375, 'center': 1.375, 'cohesion_diff': 1.125, 'threat_diff': 0.05, 'density_diff': 0.05}",
"{'diff': 1.625, 'edge': 1.5, 'center': 2.0, 'cohesion_diff': 0.625, 'threat_diff': 0.375, 'density_diff': 1.75}",
"{'diff': 2.0, 'edge': 0.625, 'center': 0.375, 'cohesion_diff': 0.625, 'threat_diff': 0.375, 'density_diff': 0.05}",
"{'diff': 0.25, 'edge': 0.125, 'center': 1.75, 'cohesion_diff': 1, 'threat_diff': 1.75, 'density_diff': 0.5}"]





#Change the values of Config_player_best and Config_player_test to numbers
data['Config_player_best'] = data['Config_player_best'].apply(lambda x: unique_values.index(x))
data['Config_player_test'] = data['Config_player_test'].apply(lambda x: unique_values.index(x))

#Print config to number
value_to_config = {}
for i in range(len(unique_values)):
    dict_config = ast.literal_eval(unique_values[i])
    #print(i, '   ------>    ', {k: round(dict_config[k]/dict_config["diff"],2) for k in dict_config})
    print(i, '   ------>    ', unique_values[i])
    value_to_config[i] = unique_values[i]

0    ------>     {'diff': 0.875, 'edge': 0.5, 'center': 2.0, 'cohesion_diff': 0.5, 'threat_diff': 0.125, 'density_diff': 1.625}
1    ------>     {'diff': 1.125, 'edge': 0.125, 'center': 1.125, 'cohesion_diff': 0.25, 'threat_diff': 0.05, 'density_diff': 0.125}
2    ------>     {'diff': 1.125, 'edge': 1.375, 'center': 1.75, 'cohesion_diff': 0.05, 'threat_diff': 0.25, 'density_diff': 1.25}
3    ------>     {'diff': 1.25, 'edge': 0.375, 'center': 1.25, 'cohesion_diff': 0.25, 'threat_diff': 1.0, 'density_diff': 0.05}
4    ------>     {'diff': 1.25, 'edge': 0.375, 'center': 1.375, 'cohesion_diff': 1.125, 'threat_diff': 0.05, 'density_diff': 0.05}
5    ------>     {'diff': 1.625, 'edge': 1.5, 'center': 2.0, 'cohesion_diff': 0.625, 'threat_diff': 0.375, 'density_diff': 1.75}
6    ------>     {'diff': 2.0, 'edge': 0.625, 'center': 0.375, 'cohesion_diff': 0.625, 'threat_diff': 0.375, 'density_diff': 0.05}
7    ------>     {'diff': 0.25, 'edge': 0.125, 'center': 1.75, 'cohesion_diff': 1, 'threat_

In [5]:
#create a dataframe with the results of the tournament
#result = 3 for a win, 1 for a draw, 0 for a loss
df = pd.DataFrame(columns=["interest_config", "comp_config", "time", "config", "white", "result" ])

for i, row in data.iterrows():
    
    config_best = row['Config_player_best']
    config_test = row['Config_player_test']
    time = row['Time']
    config = row['Config']
    winner = row['Winner']
    white = row['White']

    win_player_best = 3 if winner == "my_player_best" else 0 if winner == "my_player_test" else 1
    win_player_test = 3 if winner == "my_player_test" else 0 if winner == "my_player_best" else 1

    
    #Add a row for the player best with a dictionnary
    row_player_best = pd.DataFrame({"interest_config": [config_best, config_test], 
                                    "comp_config": [config_test, config_best], 
                                    "time": [time, time],
                                    "config": [config, config],
                                    "white": [white == "my_player_best", white == "my_player_test"]
                                    , "result": [win_player_best, win_player_test]})
    df = pd.concat([df, row_player_best], ignore_index=True)




/var/folders/n3/n6bclkp16l95kjklkk545bsr0000gn/T/ipykernel_21168/1127130516.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row_player_best], ignore_index=True)


In [24]:
def plot_victory(df, interest_config, comp_config=None, x = "config"):
    """
    Plot the victory of the interest_config against the comp_config
    """
    if comp_config is None:
        df_plot = df[df['interest_config'] == interest_config]
        name_params = "all params"
    else:
        df_plot = df[(df['interest_config'] == interest_config) & (df['comp_config'] == comp_config)]
        name_params = "param " + str(int(comp_config) + 1)

    
    score = df_plot['result'].sum()
    number_games = len(df_plot)


    #Winner plot count by result with color dict
    color_dict = {0: '#d62728', 1: '#bcbd22', 3: '#2ca02c'}
    fig = px.histogram(df_plot, x=x, color="result", barmode="group", nbins=3, color_discrete_map=color_dict, title="Victories of param Omega_" + str(int(interest_config) + 1) + " (score: " + str(score) + "/" + str(number_games) + " = " + str(round(score/number_games, 2)) + ")")
    
    #Count number of 0, 1 and 3
    count_0 = len(df_plot[df_plot['result'] == 0])
    count_1 = len(df_plot[df_plot['result'] == 1])
    count_3 = len(df_plot[df_plot['result'] == 3])

    print(f"({count_3}, {count_1}, {count_0}) : {score/number_games}")
    fig.show()
    return fig



print("Each value VS ALL")
for i in range(len(unique_values)):
    plot_victory(df, interest_config=i, comp_config = None, x = "white")
    #plot_victory(df, interest_config=i, comp_config = None, x = "white")


"""
print("\n\n\n\nthe chosen value VS each value")
chosen_config = 7
for i in range(len(unique_values)):
    if i == chosen_config:
        continue
    #plot_victory(df, interest_config=chosen_config, comp_config = i, x = "config")
    fig = plot_victory(df, interest_config=chosen_config, comp_config = i, x = "white")
"""



Each value VS ALL
(28, 19, 23) : 1.4714285714285715


(12, 32, 26) : 0.9714285714285714


(52, 12, 6) : 2.4


(14, 35, 21) : 1.1


(17, 32, 21) : 1.1857142857142857


(43, 19, 8) : 2.1142857142857143


(16, 28, 26) : 1.0857142857142856


(0, 19, 51) : 0.2714285714285714


'\nprint("\n\n\n\nthe chosen value VS each value")\nchosen_config = 7\nfor i in range(len(unique_values)):\n    if i == chosen_config:\n        continue\n    #plot_victory(df, interest_config=chosen_config, comp_config = i, x = "config")\n    fig = plot_victory(df, interest_config=chosen_config, comp_config = i, x = "white")\n'